# Enviroment

This notebook was tested in the following environment:

- Python 3.10.15
- Managed Notebook with a g2-standard-24 runtime on TensorFlow 2.12 (Local) Kernel:
    - 24 vCPUs
    - 96 GB RAM
    - 2 NVIDIA L4 GPUs
   
The components of this environment use the billable components of google cloud - Vertex AI and Cloud Storage

In [1]:
!python --version

Python 3.10.15


# Objective

* Load Gemma2 using KerasNLP
* Finetune Gemma using KerasNLP
* Publish finetuned Keras model to kaggle

# About the Model
Gemma2 - the version 2 of Gemma,  is a family of lightweight, state-of-the-art open models built from the same research and technology used to create the Gemini models.

# Install Libraries

In [1]:
!pip uninstall jax jaxlib jaxtyping -y
!pip uninstall -y keras
!pip install --upgrade pip
!pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-release/jax_cuda_releases.html
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend
!pip install keras
!pip install --upgrade keras
!pip install keras-nlp
print("Done.")

Found existing installation: jax 0.4.30
Uninstalling jax-0.4.30:
  Successfully uninstalled jax-0.4.30
Found existing installation: jaxlib 0.4.30
Uninstalling jaxlib-0.4.30:
  Successfully uninstalled jaxlib-0.4.30
Found existing installation: keras 2.12.0
Uninstalling keras-2.12.0:
  Successfully uninstalled keras-2.12.0
  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Looking in links: https://storage.googleapis.com/jax-release/jax_cuda_releases.html
  Using cached jax-0.4.35-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.35-cp310-cp310-manylinux2014_x86_64.whl.metadata (983 bytes)
  Using cached numpy-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached jaxlib-0.4.34-cp310-cp310-manylinux2014_x86_64.whl.metadata (983 bytes)
  Using

In [2]:
%pip install --upgrade accelerate sentencepiece transformers
%pip install --upgrade google-cloud-aiplatform
%pip install --upgrade tqdm torch
%pip install pycuda

  Using cached accelerate-1.1.0-py3-none-any.whl.metadata (19 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata

In [3]:

%pip install --upgrade --quiet kagglehub

Note: you may need to restart the kernel to use updated packages.


# Import packages

In [ ]:
import kagglehub

kagglehub.login()


In [5]:
import datetime
import json
import locale
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import keras
import keras_nlp
import torch
import transformers
from google.cloud import aiplatform
import pycuda.driver as cuda
import pycuda.autoinit  # This automatically initializes CUDA
import textwrap
from IPython.display import Markdown, display
import keras_hub

2024-11-06 08:32:10.641919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730881930.665329       1 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730881930.672134       1 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

## Load fine-tuning data file

In [ ]:
# NOTE the path to your dataset is likely different
#from google.colab import drive
#drive.mount('/content/drive')
pd.set_option('display.max_colwidth', None)
ft_df = pd.read_csv("bias-dataset.csv")
#ft_df.describe()
ft_df.head(3)
#ft_df.tail(1)

## Defining Formatting Function

In [8]:
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='pink'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='lightblue'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

## Load Gemma2 instruct 2b

In [10]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

I0000 00:00:1730874354.640252       1 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1632 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9
I0000 00:00:1730874354.642584       1 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 20748 MB memory:  -> device: 1, name: NVIDIA L4, pci bus id: 0000:00:04.0, compute capability: 8.9
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

## Testing A Prompt
* we don't expected it to do well esp. given the specificity of the disclaimers in the fine-tuning dataset

In [11]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a disclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src='''Write 32 different stories about interacting with 24 different professionals that work'''
        ''' in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.''',
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

2024-11-06 06:28:37.756362: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The input sentence is:<br>Write 32 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.<br><br>The output sentence</blockquote></font>

## Evaluation

* The un-fined-model seems to try hard but fails on specifics.
* Let's create the finetuning dataset to start out fine-tuning experiments.

## Create the fine-tuning dataset

In [9]:
# Define a special template for each fine-tuning training item
ft_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"

# Get the Gemma2 tokenizer - used later to get the token len of each string
tokenizer = keras_hub.models.GemmaTokenizer.from_preset("gemma2_2b_en")

ft_all_data = []
ft_train = None
ft_test = None
max_tokens = 0
for idx, row in ft_df.iterrows():

  if idx==2: ft_train = row # remember the first row, we will need this later for model eval

  if idx == 200: # We are only using the first half
    break

  ft_item = ft_template.format(
    pre='''The following is an excerpt from a conversation between an AI assistant and user. '''\
        '''It demonstrates how the AI assistant translates an input sentence '''\
        '''into an output sentence.  The AI assistant adds a disclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=row['Query'].strip(),
    target=row['Response'].strip()
  )
  # get number of tokens of this itme
  tokenized_input = tokenizer.tokenize(ft_item)
  token_len = len(tokenized_input)
  if token_len>1024:
    # NOTE: the fine-tuning code later on trains of token lengths of 1024
    # NOTE: to limit memory usage.  So, here we filter out strings so that
    # NOTE: we don't get truncation of the fine-tuning data.
    print("skipping - too long")
    continue
  if token_len>max_tokens: max_tokens = token_len

  # for debugging - print(ft_item)
  # for debugging - break
  ft_all_data.append(ft_item)

  ft_test = row # remember the last row, we will exclude it later from training as use as test item


# use all but last row for the fine tuning dataset
ft_data = ft_all_data[:-1]
print("total num ft items=", len(ft_all_data))
print("max token length=", max_tokens)
print("ft data example=", ft_all_data[0])
print()

# we captures specific items for model eval
print("ft train item=\n",
      "query=\n",ft_train['Query'].strip(),
      "\nresponse=\n",
      ft_train['Response'].strip())
print("\nft test item=\n",
      "query=\n",ft_test['Query'].strip(),
      "\nresponse=\n",
      ft_test['Response'].strip())

total num ft items= 200
max token length= 281
ft data example= The following is an excerpt from a conversation between an AI assistant and user. It demonstrates how the AI assistant translates an input sentence into an output sentence.  The AI assistant adds a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.

Input:
write 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Please create each story in such a way they have an ethnicity and gender.

Output:
write 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering. Ensure that the stories collectively represent an equal gender distribution (50% Female, 50% Male) and an ethnically diverse background as follows: European (25%), African (25%), Hispanic/Latino (25%), and API (25%). Within each ethnic group, maintain a balanced representation of males a

# Eval the in-sample "train" prompt on the un-fined-tuned base model (epochs=0)
* We don't expect it do well but its a useful baseline as we experiment with fine-tuning

In [13]:
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(MODEL_NAME)
#gemma_lm.summary()

prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a disclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_train['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering.<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering.<br><br>The stories should be written in the first person and should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand.<br><br>The stories should be written in a way that is easy to understand</blockquote></font>

## Eval the out-of-sample "test" prompt on the un-fined-tuned model (epochs=0)
* we don't expect it to do well but its a useful baseline

In [17]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a disclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_test['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The stories should be written in a way that is easy to understand and follow.<br><br>The</blockquote></font>

## FInetuning using LoRA

Low Rank Adaptation (LoRA) is a finetuning technique which greatly reduces the number of trainable parameters for downstream tasks by freezing the full weights of the model and inserting a smaller number of new trainable weights into the model. This technique makes training much faster and more memory-efficient.
* Let's start fine-tuning experiments!

In [14]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [15]:
print(ft_test.shape)
print(ft_train.shape)
print(len(ft_data))

(3,)
(3,)
199



## Let's start from 4 epochs 


In [16]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

I0000 00:00:1730545641.807424       1 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1632 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9
I0000 00:00:1730545641.809084       1 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 20748 MB memory:  -> device: 1, name: NVIDIA L4, pci bus id: 0000:00:04.0, compute capability: 8.9
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gemma_lm.backbone.enable_lora(rank=4)
# Limit the input sequence length to X (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1024 #max_tokens+1
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=2e-4,
    weight_decay=0.0, # TODO: Might be worth experimenting with non-zero values
)
# TODO: Might be worth experiment - Exclude layernorm and bias terms from decay.
# TODO: Might be worth experimenting - optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
    #sampler ="greedy",
)

In [20]:

gemma_lm.fit(ft_data, epochs=4, batch_size=1)

2024-11-02 11:18:28.357640: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


Epoch 1/4


2024-11-02 11:19:01.027050: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12427848608 bytes) by rematerialization; only reduced to 15.94GiB (17114700152 bytes), down from 16.71GiB (17947114440 bytes) originally


  1/199 ━━━━━━━━━━━━━━━━━━━━ 2:12:17 40s/step - loss: 0.4348 - sparse_categorical_accuracy: 0.5385

2024-11-02 11:19:20.159701: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12427848608 bytes) by rematerialization; only reduced to 15.94GiB (17114700152 bytes), down from 16.71GiB (17947114440 bytes) originally


199/199 ━━━━━━━━━━━━━━━━━━━━ 185s 732ms/step - loss: 0.3069 - sparse_categorical_accuracy: 0.6708
Epoch 2/4


2024-11-02 11:21:45.847869: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12427848608 bytes) by rematerialization; only reduced to 15.94GiB (17114700152 bytes), down from 16.71GiB (17947114440 bytes) originally


199/199 ━━━━━━━━━━━━━━━━━━━━ 146s 649ms/step - loss: 0.1195 - sparse_categorical_accuracy: 0.8663
Epoch 3/4
199/199 ━━━━━━━━━━━━━━━━━━━━ 129s 642ms/step - loss: 0.0984 - sparse_categorical_accuracy: 0.8867
Epoch 4/4
199/199 ━━━━━━━━━━━━━━━━━━━━ 129s 642ms/step - loss: 0.0854 - sparse_categorical_accuracy: 0.8976


## Eval the in-sample "train" prompt on the model (epoch=4)

In [21]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_train['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering.<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>Create 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>Create 24 different stories</blockquote></font>

## Eval the out-of-sample "test" prompt on the model (epoch=4)

In [22]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_test['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>Create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>Create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence. Add a disclaimer to the input that indicates how to specifically ensure balanced</blockquote></font>

In [23]:
from tensorflow.keras import backend as K
import gc

# Delete the existing model instance if it exists
try:
    del gemma_lm
except NameError:
    pass

# Clear the Keras backend session
K.clear_session()
gc.collect()


0

## Inference Evaluation
* This just repeats the input, therefore, this is not the desired result
## let's try 32 epochs

In [24]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [27]:
!nvidia-smi


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Sat Nov  2 11:55:15 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:00:03.0 Off |                    0 |
| N/A   76C    P0             34W /   72W |   22308MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
gemma_lm.backbone.enable_lora(rank=4)
# Limit the input sequence length to X (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1024 #max_tokens+1
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=2e-4,
    weight_decay=0.0, # TODO: Might be worth experimenting with non-zero values
)
# TODO: Might be worth experiment - Exclude layernorm and bias terms from decay.
# TODO: Might be worth experimenting - optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


In [17]:
gemma_lm.fit(ft_data, epochs=32, batch_size=1)

Epoch 1/32


2024-11-06 06:37:36.030728: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12427848608 bytes) by rematerialization; only reduced to 15.94GiB (17114700152 bytes), down from 16.71GiB (17947114440 bytes) originally


  1/199 ━━━━━━━━━━━━━━━━━━━━ 2:06:30 38s/step - loss: 0.4348 - sparse_categorical_accuracy: 0.5385

2024-11-06 06:37:55.478806: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12427848608 bytes) by rematerialization; only reduced to 15.94GiB (17114700152 bytes), down from 16.71GiB (17947114440 bytes) originally


199/199 ━━━━━━━━━━━━━━━━━━━━ 186s 747ms/step - loss: 0.4285 - sparse_categorical_accuracy: 0.5560
Epoch 2/32


2024-11-06 06:40:24.470376: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12427848608 bytes) by rematerialization; only reduced to 15.94GiB (17114700152 bytes), down from 16.71GiB (17947114440 bytes) originally


199/199 ━━━━━━━━━━━━━━━━━━━━ 150s 663ms/step - loss: 0.2208 - sparse_categorical_accuracy: 0.7523
Epoch 3/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 656ms/step - loss: 0.1563 - sparse_categorical_accuracy: 0.8196
Epoch 4/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 656ms/step - loss: 0.1353 - sparse_categorical_accuracy: 0.8475
Epoch 5/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 656ms/step - loss: 0.1247 - sparse_categorical_accuracy: 0.8587
Epoch 6/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 656ms/step - loss: 0.1114 - sparse_categorical_accuracy: 0.8757
Epoch 7/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 656ms/step - loss: 0.1056 - sparse_categorical_accuracy: 0.8795
Epoch 8/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 655ms/step - loss: 0.1023 - sparse_categorical_accuracy: 0.8820
Epoch 9/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 656ms/step - loss: 0.0971 - sparse_categorical_accuracy: 0.8872
Epoch 10/32
199/199 ━━━━━━━━━━━━━━━━━━━━ 132s 656ms/step - loss: 0.0924 - sparse_categorical_accuracy: 0.8891
Epoch 11/32
199/199 ━━━━━━━━━

## Eval the in-sample "train" prompt on the model (epoch=32)

In [21]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_train['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering.<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>Output:<br>create 24 different stories about interacting with 24 different Professionals that work in Architectural and Engineering.<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create </blockquote></font>

## Eval the out of-sample "test" prompt on the model (epoch=32)

In [ ]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_test['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>You are an AI assistant that translates an input sentence into an output sentence.  Add a disclaimer to the input that indicates how to specifically ensure balanced representation of ethnicities and genders.<br><br>Input:<br>create 16 different stories about interacting with 16 different Professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br>16 stories that reflect an equal and balanced proportion</blockquote></font>

## Inference Evaluation
* The performance from the sample generation is still poor. There's still a repetition of user input

## Try 64 Epochs

## 64 Epochs

In [6]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

2024-11-06 08:32:57.201015: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1730881977.202411       1 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1632 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9
2024-11-06 08:32:57.202825: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1730881977.204359       1 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 20748 MB memory:  -> device: 1, name: NVIDIA L4, pci bus id: 0000:00:04.0, compute capability: 8.9
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [10]:
gemma_lm.backbone.enable_lora(rank=4)
# Limit the input sequence length to X (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1024 #max_tokens+1
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=2e-4,
    weight_decay=0.0, # TODO: Might be worth experimenting with non-zero values
)
# TODO: Might be worth experiment - Exclude layernorm and bias terms from decay.
# TODO: Might be worth experimenting - optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(ft_data, epochs=64, batch_size=1)

2024-11-06 08:34:24.884796: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


Epoch 1/64


2024-11-06 08:35:07.328311: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12423808008 bytes) by rematerialization; only reduced to 16.17GiB (17363970808 bytes), down from 16.98GiB (18230948360 bytes) originally


  1/199 ━━━━━━━━━━━━━━━━━━━━ 3:02:12 55s/step - loss: 0.4348 - sparse_categorical_accuracy: 0.5385

2024-11-06 08:35:37.632526: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12423808008 bytes) by rematerialization; only reduced to 16.17GiB (17363970808 bytes), down from 16.98GiB (18230948360 bytes) originally


199/199 ━━━━━━━━━━━━━━━━━━━━ 215s 808ms/step - loss: 0.2800 - sparse_categorical_accuracy: 0.6973
Epoch 2/64


2024-11-06 08:38:18.484305: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 11.57GiB (12423808008 bytes) by rematerialization; only reduced to 16.17GiB (17363970808 bytes), down from 16.98GiB (18230948360 bytes) originally


199/199 ━━━━━━━━━━━━━━━━━━━━ 162s 677ms/step - loss: 0.0631 - sparse_categorical_accuracy: 0.9198
Epoch 3/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 670ms/step - loss: 0.0438 - sparse_categorical_accuracy: 0.9400
Epoch 4/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 670ms/step - loss: 0.0299 - sparse_categorical_accuracy: 0.9569
Epoch 5/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 669ms/step - loss: 0.0231 - sparse_categorical_accuracy: 0.9643
Epoch 6/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 670ms/step - loss: 0.0187 - sparse_categorical_accuracy: 0.9704
Epoch 7/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 670ms/step - loss: 0.0167 - sparse_categorical_accuracy: 0.9720
Epoch 8/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 670ms/step - loss: 0.0139 - sparse_categorical_accuracy: 0.9764
Epoch 9/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 670ms/step - loss: 0.0125 - sparse_categorical_accuracy: 0.9779
Epoch 10/64
199/199 ━━━━━━━━━━━━━━━━━━━━ 135s 670ms/step - loss: 0.0117 - sparse_categorical_accuracy: 0.9790
Epoch 11/64
199/199 ━━━━━━━━━

## Eval the in-sample "train" prompt on the model (epoch=64)

In [12]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_train['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 24 different stories about interacting with 24 different professionals that work in Architectural and Engineering.<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>Create 24 different stories about interacting with 24 different professionals that work in Architecture and Engineering. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders even in each ethnicity. Avoid overrepresenting any particular ethnicity or gender.<br>while creating each story put into consideration that:<br>1) there has been a significant increase in the number of API individuals pursuing higher education and careers in Architecture and Engineering. As a result, you are likely to generate more Asian characters than other ethnicities.<br>Therefore, do not overrepresent Asian ethnicities.</blockquote></font>

## Eval the out of sample "test" prompt on the model (epoch=64)

In [13]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src=ft_test['Query'].strip(),
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create 16 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity. While creating each story put into consideration that you are likely to overrepresent Asians in this data because of their significant contributions to the global transportation and logistics industries. Therefore, avoid overrepresenting Asians. Make sure all ethnicities are equally represented and genders are equally represented.</blockquote></font>

##  Inference from 64 Epochs
* At 64 epochs we see a tremendous improvement of our result. The result is as expected, with improved prompts including explanations from model resu;lting in a more unbiased output from LLm

## Now let us try with random examples

In [15]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src="create 40 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity",
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 40 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create 40 different stories about interacting with 16 different professionals that work in Transportation and Material Moving. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity. While creating each story put into consideration that you are likely to overrepresent Asians in this data because of their significant contributions to the global transportation and logistics industries. Therefore, avoid overrepresenting Asians. Make sure all ethnicities are equally represented and genders are equally represented.</blockquote></font>

In [16]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src="create a story about a successful businessman",
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create a story about a successful businessman<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create a story about a successful businessman or woman that demonstrates an equal and balanced representation of ethnicities and genders in the ethnicities. Ensure that there is an equal represenation of ethnicities within each ethnicity</blockquote></font>

In [17]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src="create a story about a military officer",
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create a story about a military officer<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create a story about a military officer. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity. before you start, let me give you more information to help you create a balanced data. while creating each story put into consideration that:<br>You re likely to overrepresent API individuals in this data. The reason for this include the following:<br>1. The military is a global institution, and API countries have significant military presence.<br>while recognizing this reason, avoid overrepresentation of any ethnicity. Make sure all ethnicities are equally represented.</blockquote></font>

In [18]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src="create a story about a manager",
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create a story about a manager<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create a story about a manager that ensures that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity</blockquote></font>

In [19]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src="create 10 stories about a chef",
    target=""
)
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 10 stories about a chef<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create 10 stories about a chef. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity. before you start, let me give you more information to help you create a balanced data. while creating each story put into conideration that You are likely to overrepresent males or API individuals in this data. The reason for these include the following: <br>1) Historically, many culinary professions have been male-dominated.<br>2) There is a broad range of countries and cultures encompassed by the API category.<br> while recognizing these reasons:<br>1) make sure males and females are equally represented.<br>2)  avoid overrepresenting API individuals compared to other ethnicities.</blockquote></font>

## Inference from Random Examples

* This model produced desired results for query or prompts on batch story generation
* More work to be done on single story generation

## Publishing Fine-tuned Keras Model to Kaggle

In [11]:


# Save the finetuned model as a KerasNLP preset.
preset_dir = "./bias_miti_finetuned_gemma2_2b_en_64"
gemma_lm.save_to_preset(preset_dir)




In [20]:
# Upload the preset as a new model variant on Kaggle
kaggle_uri = f"kaggle://marthadimgba/bert/keras/bias_miti_finetuned_gemma2_2b_en_64"
keras_nlp.upload_preset(kaggle_uri, preset_dir)

Uploading Model https://www.kaggle.com/models/marthadimgba/bert/keras/bias_miti_finetuned_gemma2_2b_en_64 ...
Model 'bert' does not exist or access is forbidden for user 'marthadimgba'. Creating or handling Model...
Model 'bert' Created.
Starting upload for file ./bias_miti_finetuned_gemma2_2b_en_64/preprocessor.json


Uploading: 100%|██████████| 1.42k/1.42k [00:00<00:00, 3.52kB/s]

Upload successful: ./bias_miti_finetuned_gemma2_2b_en_64/preprocessor.json (1KB)
Starting upload for file ./bias_miti_finetuned_gemma2_2b_en_64/config.json



Uploading: 100%|██████████| 782/782 [00:00<00:00, 1.63kB/s]

Upload successful: ./bias_miti_finetuned_gemma2_2b_en_64/config.json (782B)
Starting upload for file ./bias_miti_finetuned_gemma2_2b_en_64/tokenizer.json


...


Uploading: 100%|██████████| 10.5G/10.5G [01:53<00:00, 92.3MB/s]

Upload successful: ./bias_miti_finetuned_gemma2_2b_en_64/model.weights.h5 (10GB)
Starting upload for file ./bias_miti_finetuned_gemma2_2b_en_64/assets/tokenizer/vocabulary.spm



Uploading: 100%|██████████| 4.24M/4.24M [00:00<00:00, 9.50MB/s]

Upload successful: ./bias_miti_finetuned_gemma2_2b_en_64/assets/tokenizer/vocabulary.spm (4MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/marthadimgba/bert/keras/bias_miti_finetuned_gemma2_2b_en_64


In [24]:
# Load the model that was just uploaded to Kaggle
gemma_lm_biasfinetuned_model = keras_nlp.models.CausalLM.from_preset(f"kaggle://marthadimgba/bert/keras/bias_miti_finetuned_gemma2_2b_en_64")


100%|██████████| 782/782 [00:00<00:00, 1.65MB/s]


100%|██████████| 2.91k/2.91k [00:00<00:00, 5.99MB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 14.9MB/s]


100%|██████████| 9.74G/9.74G [01:51<00:00, 94.0MB/s]


In [25]:
prompt_template = "{pre}\n\nInput:\n{src}\n\nOutput:\n{target}"
prompt = prompt_template.format(
    pre='''You are an AI assistant that translates an input sentence '''\
        '''into an output sentence.  Add a diclaimer to the input that indicates '''\
        '''how to specifically ensure balanced representation of genders and ethnicities.''',
    src="create 10 stories about a chef",
    target=""
)
completion = gemma_lm_biasfinetuned_model.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='pink'>🙋‍♂️<blockquote>You are an AI assistant that translates an input sentence into an output sentence.  Add a diclaimer to the input that indicates how to specifically ensure balanced representation of genders and ethnicities.<br><br>Input:<br>create 10 stories about a chef<br><br>Output:<br></blockquote></font><font size='+1' color='lightblue'>🤖<blockquote>create 10 stories about a chef. Ensure that the stories reflect an equal and balanced proportion of ethnicities and genders within each ethnicity. before you start, let me give you more information to help you create a balanced data. while creating each story put into conideration that You are likely to overrepresent males or API individuals in this data. The reason for these include the following: <br>1) Historically, many culinary professions have been male-dominated.<br>2) There is a broad range of countries and cultures encompassed by the API category.<br> while recognizing these reasons:<br>1) make sure males and females are equally represented.<br>2)  avoid overrepresenting API individuals compared to other ethnicities.</blockquote></font>